# 02 GSAT Metrics #

In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/home/563/sc1326/repos/cdrmip_extremes')
from cdrmip_extremes.configs import data_dir, models, expts
from cdrmip_extremes import load_data
from cdrmip_extremes.utils import global_mean, calc_anomaly, concat_branches
from cdrmip_extremes.sat import find_crossing_years, find_matching_gwls

In [2]:
client = Client(memory_limit=None,threads_per_worker=1,n_workers=28)

### Load GSAT ###

In [3]:
gsat = load_data.load_gsat()

In [4]:
# # convert each gsat dataarray to a dataset
# for model, da in gsat.items():
#     gsat[model] = da.to_dataset(name='gsat')

# # then create datatree
# gsat_dt = xr.DataTree.from_dict(gsat)
# gsat_dt

### Calculate relevant metrics ###
Peak warming, warming rate, cooling_rate, warming at return to PI concentrations, finaly 21-year anomaly

In [5]:
def calc_gsat_metrics(ds):
    if 'year' not in ds.dims:
        ds = ds.groupby('time.year').mean(dim='time')
    ds_rolling = ds.rolling(year=21,center=True,min_periods=1).mean()
    peak = ds_rolling.max(dim='year')
    peak_year = ds_rolling.idxmax(dim='year')
    peak_co2_warming = ds_rolling.sel(year=140)
    return_to_PI_anom = ds_rolling.sel(year=280)
    metric_dict = {
        'Peak warming (°C)': np.round(peak.values,2),
        'Year of peak warming': peak_year.values,
        'Warming rate to peak warming (°C/decade)': np.round(((peak/peak_year)*10).values,2),
        'Warming at peak CO₂ (°C)': np.round(peak_co2_warming.values,2),
        'Warming rate to peak CO₂ (°C/decade)': np.round(((peak_co2_warming/140)*10).values,2),
        'Cooling rate from peak warming (°C/decade)': np.round((10*(return_to_PI_anom-peak)/(280-peak_year)).values,2),
        'Anomaly upon return to PI concentrations (°C)': np.round(return_to_PI_anom.values,2),
        'Final 21-year anomaly (°C)': np.round(ds_rolling.isel(year=slice(320,340)).mean(dim='year').values,2),
    }
    
    return metric_dict

In [6]:
metric_dicts = {}
for model, ds in gsat.items():
    metric_dicts[model] = calc_gsat_metrics(ds)
metric_dicts

{'ACCESS-ESM1-5': {'Peak warming (°C)': 4.4,
  'Year of peak warming': array(152.),
  'Warming rate to peak warming (°C/decade)': 0.29,
  'Warming at peak CO₂ (°C)': 4.31,
  'Warming rate to peak CO₂ (°C/decade)': 0.31,
  'Cooling rate from peak warming (°C/decade)': -0.27,
  'Anomaly upon return to PI concentrations (°C)': 0.92,
  'Final 21-year anomaly (°C)': 0.64},
 'CanESM5': {'Peak warming (°C)': 6.65,
  'Year of peak warming': array(150.),
  'Warming rate to peak warming (°C/decade)': 0.44,
  'Warming at peak CO₂ (°C)': 6.45,
  'Warming rate to peak CO₂ (°C/decade)': 0.46,
  'Cooling rate from peak warming (°C/decade)': -0.39,
  'Anomaly upon return to PI concentrations (°C)': 1.56,
  'Final 21-year anomaly (°C)': 0.9},
 'CESM2': {'Peak warming (°C)': 5.35,
  'Year of peak warming': array(150.),
  'Warming rate to peak warming (°C/decade)': 0.36,
  'Warming at peak CO₂ (°C)': 5.21,
  'Warming rate to peak CO₂ (°C/decade)': 0.37,
  'Cooling rate from peak warming (°C/decade)': -0.

In [7]:
global_df = pd.DataFrame.from_dict(metric_dicts,orient='columns')

In [8]:
ECS = {
    'ACCESS-ESM1-5':3.87,
    'CanESM5':5.63,
    'CESM2':5.18,
    'CNRM-ESM2-1':4.76,
    'GFDL-ESM4':2.62,
    'MIROC-ES2L':2.68,
    'NorESM2-LM':2.54,
    'UKESM1-0-LL':5.34
}

global_df.loc['ECS from Song et al. (2023) (°C)'] = ECS

In [9]:
global_df

,ACCESS-ESM1-5,CanESM5,CESM2,CNRM-ESM2-1,GFDL-ESM4,MIROC-ES2L,NorESM2-LM,UKESM1-0-LL
Peak warming (°C),4.4,6.65,5.35,5.46,3.76,3.65,3.47,6.71
Year of peak warming,152.0,150.0,150.0,147.0,150.0,145.0,148.0,149.0
Warming rate to peak warming (°C/decade),0.29,0.44,0.36,0.37,0.25,0.25,0.23,0.45
Warming at peak CO₂ (°C),4.31,6.45,5.21,5.3,3.58,3.59,3.45,6.56
Warming rate to peak CO₂ (°C/decade),0.31,0.46,0.37,0.38,0.26,0.26,0.25,0.47
Cooling rate from peak warming (°C/decade),-0.27,-0.39,-0.31,-0.32,-0.24,-0.24,-0.24,-0.35
Anomaly upon return to PI concentrations (°C),0.92,1.56,1.26,1.17,0.69,0.36,0.34,2.17
Final 21-year anomaly (°C),0.64,0.9,1.27,0.81,0.55,0.23,0.6,1.51
ECS from Song et al. (2023) (°C),3.87,5.63,5.18,4.76,2.62,2.68,2.54,5.34


In [10]:
global_df.loc['Warming rate to peak CO₂/ Cooling Rate from peak CO₂'] = (
    global_df.loc['Warming rate to peak CO₂ (°C/decade)'] / (-global_df.loc['Cooling rate from peak warming (°C/decade)'])
)

In [11]:
global_df

,ACCESS-ESM1-5,CanESM5,CESM2,CNRM-ESM2-1,GFDL-ESM4,MIROC-ES2L,NorESM2-LM,UKESM1-0-LL
Peak warming (°C),4.4,6.65,5.35,5.46,3.76,3.65,3.47,6.71
Year of peak warming,152.0,150.0,150.0,147.0,150.0,145.0,148.0,149.0
Warming rate to peak warming (°C/decade),0.29,0.44,0.36,0.37,0.25,0.25,0.23,0.45
Warming at peak CO₂ (°C),4.31,6.45,5.21,5.3,3.58,3.59,3.45,6.56
Warming rate to peak CO₂ (°C/decade),0.31,0.46,0.37,0.38,0.26,0.26,0.25,0.47
Cooling rate from peak warming (°C/decade),-0.27,-0.39,-0.31,-0.32,-0.24,-0.24,-0.24,-0.35
Anomaly upon return to PI concentrations (°C),0.92,1.56,1.26,1.17,0.69,0.36,0.34,2.17
Final 21-year anomaly (°C),0.64,0.9,1.27,0.81,0.55,0.23,0.6,1.51
ECS from Song et al. (2023) (°C),3.87,5.63,5.18,4.76,2.62,2.68,2.54,5.34
Warming rate to peak CO₂/ Cooling Rate from peak CO₂,1.148148,1.179487,1.193548,1.1875,1.083333,1.083333,1.041667,1.342857
